# [자연어처리]
# 11주차(11-2). 언어 모델과 NNLM

# 202002961 김현주
* **모든 셀 실행** 후 제출하시기 바랍니다.
* **실습 (11-2-1)**이 있습니다.

---

In [1]:
from collections import Counter

In [2]:
# 코퍼스 정의
corpus = [
    "I eat a strawberry",
    "I eat a blueberry",
    "I eat a strawberry cake"
]

---

# 통계 기반의 방법 활용 (10주차)
## 1-1. bigram (2-gram) 확률 계산

In [3]:
# 바이그램 모델 구축
def train_bigram_model(corpus):
    words = [word for sentence in corpus for word in sentence.split()]
    unigrams = Counter(words)
    bigrams = Counter(zip(words, words[1:]))

    # 바이그램 확률 계산 (스무딩 없이)
    bigram_probabilities = {}
    for bigram, count in bigrams.items():
        bigram_probabilities[bigram] = count / unigrams[bigram[0]]

    return bigram_probabilities, unigrams

In [4]:
# 모델 학습 (스무딩 없이)
bigram_probabilities, unigrams = train_bigram_model(corpus)

## P(cake|blueberry)를 계산해보자!
- 단순 카운트(방법 1-1)를 이용한 계산

In [5]:
# 사용자로부터 단어 입력받아 확률 계산
w1 = 'blueberry' # input("Enter the first word: ")
w2 = 'cake' # input("Enter the second word: ")
bigram = (w1, w2)

# 확률 조회
if bigram in bigram_probabilities:
    probability = bigram_probabilities[bigram]
else:
    # 스무딩 없이 확률을 0으로 처리
    probability = 0

print(f"P({w2} | {w1}) without Laplace Smoothing is: {probability}")

P(cake | blueberry) without Laplace Smoothing is: 0


'blueberry cake'라는 bigram이 corpus에 존재하지 않기 때문에, 스무딩과 같은 방법을 적용하지 않을 경우 값이 0이됨.

---

## 1-2. 코퍼스에 등장하지 않았던 bigram을 계산하는 방법
- 라플라스 스무딩 적용

In [ ]:
# 중복된 코드이므로 주석 처리됨
# from collections import Counter, defaultdict

In [ ]:
# 중복된 코드이므로 주석 처리됨
# 코퍼스 정의
# corpus = [
#     "I eat a strawberry",
#     "I eat a blueberry",
#     "I eat a strawberry cake"
# ]

In [6]:
# 바이그램 모델 구축
def train_bigram_model(corpus, alpha=1, use_smoothing=True):
    words = [word for sentence in corpus for word in sentence.split()]
    unigrams = Counter(words)
    bigrams = Counter(zip(words, words[1:]))
    vocabulary_size = len(unigrams)

    # 바이그램 확률 계산
    bigram_probabilities = {}
    for bigram, count in bigrams.items():
        if use_smoothing:
            bigram_probabilities[bigram] = (count + alpha) / (unigrams[bigram[0]] + alpha * vocabulary_size)
        else:
            bigram_probabilities[bigram] = count / unigrams[bigram[0]]

    return bigram_probabilities, unigrams, vocabulary_size

In [7]:
# 모델 학습 (라플라스 스무딩 사용 여부 선택 가능)
use_smoothing = "yes" # input("Use Laplace Smoothing? (yes/no): ").strip().lower() == "yes"
bigram_probabilities, unigrams, vocabulary_size = train_bigram_model(corpus, use_smoothing=use_smoothing)

## P(cake|blueberry)를 계산해보자!
- 단순 카운트를 이용한 계산 + 라플라스 스무딩(방법 1-2)을 이용한 보정

In [8]:
# 사용자로부터 단어 입력받아 확률 계산
w1 = 'blueberry' # input("Enter the first word: ")
w2 = 'cake' # input("Enter the second word: ")
bigram = (w1, w2)

# 확률 조회
if bigram in bigram_probabilities:
    probability = bigram_probabilities[bigram]
else:
    # 라플라스 스무딩을 사용하는 경우 기본 확률을 스무딩된 값으로, 그렇지 않으면 0으로 처리
    if use_smoothing:
        probability = 1 / (unigrams.get(w1, 0) + vocabulary_size)
    else:
        probability = 0

print(f"P({w2} | {w1}) with{'out' if not use_smoothing else ''} Laplace Smoothing is: {probability}")

P(cake | blueberry) with Laplace Smoothing is: 0.14285714285714285


---

# 신경망 기반의 방법 활용 (11주차)
## 2-1. NNLM을 이용한 bigram 계산

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
from torch.utils.data import Dataset, DataLoader

# 난수 시드 고정
torch.manual_seed(42)

In [10]:
# 중복된 코드이므로 주석 처리됨
# 코퍼스 정의
# corpus = [
#     "I eat a strawberry",
#     "I eat a blueberry",
#     "I eat a strawberry cake"
# ]

In [11]:
# 단어 토큰화 및 단어 사전 구축
words = [word for sentence in corpus for word in sentence.split()]
vocab = list(set(words))
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for word, i in word_to_idx.items()}

In [12]:
# Hyperparameters
embedding_dim = 10
hidden_dim = 10
batch_size = 2
learning_rate = 0.01
epochs = 200
context_size = 1  # Bigram을 위해 context_size를 1로 설정

In [13]:
# Dataset 및 DataLoader 생성
class NgramDataset(Dataset):
    def __init__(self, corpus, context_size):
        self.ngrams = []
        for sentence in corpus:
            words = sentence.split()
            for i in range(len(words) - context_size):
                context = words[i:i + context_size]
                target = words[i + context_size]
                context_idxs = [word_to_idx[w] for w in context]
                target_idx = word_to_idx[target]
                self.ngrams.append((context_idxs, target_idx))

    def __len__(self):
        return len(self.ngrams)

    def __getitem__(self, idx):
        return torch.tensor(self.ngrams[idx][0]), torch.tensor(self.ngrams[idx][1])

In [14]:
dataset = NgramDataset(corpus, context_size)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

각 레이어 설명
- Input Layer: 단어 인덱스를 임베딩 벡터로 변환하여 입력받음.
- Projection Layer: 여러 단어의 임베딩을 결합한 후, 이를 선형 변환하여 차원을 조정.
- Hidden Layer: 비선형 활성화 함수 (tanh)를 적용하여 모델이 더 복잡한 패턴을 학습함.
- Output Layer: 최종적으로 예측된 단어의 확률 분포를 출력.

In [15]:
# Neural Network Language Model 정의
class NNLM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, context_size=1):
        super(NNLM, self).__init__()

        # 임베딩 레이어 (Input Layer)
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)

        # Projection Layer (Linear Transformation)
        self.projection = nn.Linear(embedding_dim * context_size, hidden_dim)

        # Hidden Layer (Nonlinear Transformation with tanh)
        self.hidden = nn.Linear(hidden_dim, hidden_dim)

        # Output Layer
        self.output = nn.Linear(hidden_dim, vocab_size)

    def forward(self, context_words):
        # Input Layer: 여러 단어 임베딩 후, 결합
        emb = self.embeddings(context_words)  # context_words의 형태는 (batch_size, context_size)
        emb = emb.view(emb.size(0), -1)       # (batch_size, embedding_dim * context_size)

        # Projection Layer: 선형 변환
        proj = self.projection(emb)

        # Hidden Layer: 비선형 활성화 함수 tanh 적용
        hidden_out = torch.tanh(self.hidden(proj))

        # Output Layer: 다음 단어에 대한 확률 분포 계산
        out = self.output(hidden_out)  # (batch_size, vocab_size)

        return out

In [16]:
# 모델, 손실 함수, 옵티마이저 초기화
vocab_size = len(vocab)
model = NNLM(vocab_size, embedding_dim, hidden_dim, context_size) # bigram=1
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
# 모델 학습
for epoch in range(epochs):
    total_loss = 0
    for context, target in dataloader:
        context = context.long()
        target = target.long()

        optimizer.zero_grad()
        outputs = model(context)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

Epoch 50/200, Loss: 1.0332
Epoch 100/200, Loss: 1.0133
Epoch 150/200, Loss: 0.9821
Epoch 200/200, Loss: 0.9674


## P(cake|blueberry)를 계산해보자!
- NNLM을 이용하여 bigram 계산 (방법 2-1)

In [18]:
# 사용자로부터 단어 입력받아 확률 계산
context_words = 'blueberry'.split() # input(f"Enter the {context_size} context words separated by space: ").split()
target_word = 'cake' # input("Enter the target word: ")

# 입력 단어가 사전에 존재하는지 확인
if all(word in word_to_idx for word in context_words) and target_word in word_to_idx:
    with torch.no_grad():
        context_idxs = torch.tensor([word_to_idx[word] for word in context_words]).unsqueeze(0)
        outputs = model(context_idxs)
        probabilities = torch.softmax(outputs, dim=1)
        target_idx = word_to_idx[target_word]
        probability = probabilities[0][target_idx].item()
        print(f"P({target_word} | {' '.join(context_words)}) with NNLM is: {probability}")
else:
    print("One or more input words are not in the vocabulary.")

P(cake | blueberry) with NNLM is: 0.0024459303822368383


## P(cake|redberry)를 계산해보자!
- 'blueberry' -> 'redberry'로 변경할 경우, 'redberry'는 corpus에 없는 단어이므로, 값이 0이됨.

In [19]:
# 사용자로부터 단어 입력받아 확률 계산
context_words = 'redberry'.split() # input(f"Enter the {context_size} context words separated by space: ").split()
target_word = 'cake' # input("Enter the target word: ")

# 입력 단어가 사전에 존재하는지 확인
if all(word in word_to_idx for word in context_words) and target_word in word_to_idx:
    with torch.no_grad():
        context_idxs = torch.tensor([word_to_idx[word] for word in context_words]).unsqueeze(0)
        outputs = model(context_idxs)
        probabilities = torch.softmax(outputs, dim=1)
        target_idx = word_to_idx[target_word]
        probability = probabilities[0][target_idx].item()
        print(f"P({target_word} | {' '.join(context_words)}) with NNLM is: {probability}")
else:
    print("One or more input words are not in the vocabulary.")

One or more input words are not in the vocabulary.


---

## 2-2. 코퍼스에 등장하지 않았던 bigram을 계산하는 방법
- UNK (Unknwon 토큰) 적용

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# 난수 시드 고정
torch.manual_seed(42)

# 코퍼스 정의
corpus = [
    "I eat a strawberry",
    "I eat a blueberry",
    "I eat a strawberry cake"
]

# 단어 토큰화 및 단어 사전 구축
words = [word for sentence in corpus for word in sentence.split()]
vocab = list(set(words))
vocab.append("UNK")  # UNK 토큰 추가
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for word, i in word_to_idx.items()}
unk_idx = word_to_idx["UNK"]

# Hyperparameters
embedding_dim = 10
hidden_dim = 10
batch_size = 2
learning_rate = 0.01
epochs = 200
context_size = 1  # n-gram NNLM을 위해 context_size를 n-1로 설정 (예: trigram의 경우 2)

# Dataset 및 DataLoader 생성
class NgramDataset(Dataset):
    def __init__(self, corpus, context_size):
        self.ngrams = []
        for sentence in corpus:
            words = sentence.split()
            for i in range(len(words) - context_size):
                context = words[i:i + context_size]
                target = words[i + context_size]
                context_idxs = [word_to_idx.get(w, unk_idx) for w in context]
                target_idx = word_to_idx.get(target, unk_idx)
                self.ngrams.append((context_idxs, target_idx))

    def __len__(self):
        return len(self.ngrams)

    def __getitem__(self, idx):
        return torch.tensor(self.ngrams[idx][0]), torch.tensor(self.ngrams[idx][1])

dataset = NgramDataset(corpus, context_size)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Neural Network Language Model 정의
class NNLM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, context_size):
        super(NNLM, self).__init__()

        # 임베딩 레이어 (Input Layer)
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)

        # Projection Layer (Linear Transformation)
        self.projection = nn.Linear(embedding_dim * context_size, hidden_dim)

        # Hidden Layer (Nonlinear Transformation with tanh)
        self.hidden = nn.Linear(hidden_dim, hidden_dim)

        # Output Layer
        self.output = nn.Linear(hidden_dim, vocab_size)

    def forward(self, context_words):
        # Input Layer: 여러 단어 임베딩 후, 결합
        emb = self.embeddings(context_words)  # context_words의 형태는 (batch_size, context_size)
        emb = emb.view(emb.size(0), -1)       # (batch_size, embedding_dim * context_size)

        # Projection Layer: 선형 변환
        proj = self.projection(emb)

        # Hidden Layer: 비선형 활성화 함수 tanh 적용
        hidden_out = torch.tanh(self.hidden(proj))

        # Output Layer: 다음 단어에 대한 확률 분포 계산
        out = self.output(hidden_out)  # (batch_size, vocab_size)

        return out

# 모델, 손실 함수, 옵티마이저 초기화
vocab_size = len(vocab)
model_unk = NNLM(vocab_size, embedding_dim, hidden_dim, context_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_unk.parameters(), lr=learning_rate)

# 모델 학습
for epoch in range(epochs):
    total_loss = 0
    for context, target in dataloader:
        context = context.long()
        target = target.long()

        optimizer.zero_grad()
        outputs = model_unk(context)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

Epoch 50/200, Loss: 1.0212
Epoch 100/200, Loss: 0.9894
Epoch 150/200, Loss: 0.9687
Epoch 200/200, Loss: 0.9700


In [21]:
# 사용자로부터 단어 입력받아 확률 계산
context_words = 'redberry'.split() # input(f"Enter the {context_size} context words separated by space: ").split()
target_word = 'cake' # input("Enter the target word: ")

# 입력 단어가 사전에 존재하지 않으면 UNK 인덱스 사용
context_idxs = [word_to_idx.get(word, unk_idx) for word in context_words]
target_idx = word_to_idx.get(target_word, unk_idx)

with torch.no_grad():
    context_idxs = torch.tensor(context_idxs).unsqueeze(0)
    outputs = model_unk(context_idxs)
    probabilities = torch.softmax(outputs, dim=1)
    probability = probabilities[0][target_idx].item()
    print(f"P({target_word} | {' '.join(context_words)}) with NNLM is: {probability}")

P(cake | redberry) with NNLM is: 0.019036589190363884


---

# **[[실습 11-2-1]]** 통계 기반 방법과 신경망 기반의 방법 예시 만들기
- (아래와 같은 코퍼스가 있을 때) **코퍼스에 등장하지 않은 두 단어(bigram)**가 포함되어 있는 경우를 살펴볼 수 있는 예시를 만들어 보시오.
- 참고 사항
  - 위에서는 1-1 vs 1-2를 비교하기 위해, **'blueberry cake'**라는 예제를 살펴보았고,
  - 2-1 vs 2-2를 비교하기 위해, **'redberry cake'**라는 예제를 살펴보았음.
  - 이와 같이 (위에서 살펴본 것과 다른) 두 단어로 이루어진 예시(bigram)를 적절히 만들어 보고 코드를 실행하시오.

```
# 코퍼스 정의
corpus = [
    "I eat a strawberry",
    "I eat a blueberry",
    "I eat a strawberry cake"
]
```

### 다음의 코드에서 *'???' ### 이 부분을 변경해 보시오.* <- **???** 부분을 변경하여 코드를 실행하면 됨.


## 1-1 방법
- 통계 기반의 방법, 라플라스 스무딩 적용 X
- 예상되는 코드 실행 결과: P(w2 | w1) without Laplace Smoothing is: 0

In [24]:
use_smoothing = False

# 사용자로부터 단어 입력받아 확률 계산
w1 = 'strawberry' ### 이 부분을 변경해 보시오. ex) bluberry
w2 = 'jelly' ### 이 부분을 변경해 보시오. ex) cake
bigram = (w1, w2)

# 확률 조회
if bigram in bigram_probabilities:
    probability = bigram_probabilities[bigram]
else:
    # 라플라스 스무딩을 사용하는 경우 기본 확률을 스무딩된 값으로, 그렇지 않으면 0으로 처리
    if use_smoothing:
        probability = 1 / (unigrams.get(w1, 0) + vocabulary_size)
    else:
        probability = 0

print(f"P({w2} | {w1}) with{'out' if not use_smoothing else ''} Laplace Smoothing is: {probability}")

P(jelly | strawberry) without Laplace Smoothing is: 0


## 1-2 방법
- 통계 기반의 방법, 라플라스 스무딩 적용 O
- 예상되는 코드 실행 결과: P(w2 | w1) with Laplace Smoothing is: (확률 값)



In [25]:
use_smoothing = True

# 사용자로부터 단어 입력받아 확률 계산
w1 = 'strawberry' ### 이 부분을 변경해 보시오. ex) bluberry
w2 = 'jelly' ### 이 부분을 변경해 보시오. ex) cake
bigram = (w1, w2)

# 확률 조회
if bigram in bigram_probabilities:
    probability = bigram_probabilities[bigram]
else:
    # 라플라스 스무딩을 사용하는 경우 기본 확률을 스무딩된 값으로, 그렇지 않으면 0으로 처리
    if use_smoothing:
        probability = 1 / (unigrams.get(w1, 0) + vocabulary_size)
    else:
        probability = 0

print(f"P({w2} | {w1}) with{'out' if not use_smoothing else ''} Laplace Smoothing is: {probability}")

P(jelly | strawberry) with Laplace Smoothing is: 0.125


## 2-1 방법
- 신경망 기반의 방법, UNK 토큰 적용 X
- 예상되는 코드 실행 결과: One or more input words are not in the vocabulary.

In [26]:
# 사용자로부터 단어 입력받아 확률 계산
context_words = 'raspberry'.split() ### 이 부분을 변경해 보시오. ex) redberry
target_word = 'jelly' ### 이 부분을 변경해 보시오. ex) cake

# 입력 단어가 사전에 존재하는지 확인
if all(word in word_to_idx for word in context_words) and target_word in word_to_idx:
    with torch.no_grad():
        context_idxs = torch.tensor([word_to_idx[word] for word in context_words]).unsqueeze(0)
        outputs = model(context_idxs)
        probabilities = torch.softmax(outputs, dim=1)
        target_idx = word_to_idx[target_word]
        probability = probabilities[0][target_idx].item()
        print(f"P({target_word} | {' '.join(context_words)}) with NNLM is: {probability}")
else:
    print("One or more input words are not in the vocabulary.")

One or more input words are not in the vocabulary.


## 2-2 방법
- 신경망 기반의 방법, UNK 토큰 적용 O
- 예상되는 코드 실행 결과: P(target_word | context_words) with NNLM is: (확률 값)


In [29]:
# 사용자로부터 단어 입력받아 확률 계산
context_words = 'raspberry'.split() ### 이 부분을 변경해 보시오, ex) redberry
target_word = 'jelly' ### 이 부분을 변경해 보시오. ex) cake

# 입력 단어가 사전에 존재하지 않으면 UNK 인덱스 사용
context_idxs = [word_to_idx.get(word, unk_idx) for word in context_words]
target_idx = word_to_idx.get(target_word, unk_idx)

with torch.no_grad():
    context_idxs = torch.tensor(context_idxs).unsqueeze(0)
    outputs = model_unk(context_idxs)
    probabilities = torch.softmax(outputs, dim=1)
    probability = probabilities[0][target_idx].item()
    print(f"P({target_word} | {' '.join(context_words)}) with NNLM is: {probability}")

P(jelly | raspberry) with NNLM is: 0.0014913304476067424


---

# [파일] -> [다운로드] -> [.ipynb 다운로드]